In [ ]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Expression,
    value,
    check_optimal_termination,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)
from pyomo.network import Arc

# Imports from IDAES
from idaes.core import FlowsheetBlock, UnitModelCostingBlock
from idaes.models.unit_models import Feed, Product
from idaes.models.unit_models.mixer import (
    Mixer,
    MomentumMixingType,
)
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from idaes.core.util.initialization import propagate_state

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.unit_models.zero_order import ChemicalAdditionZO
from watertap.unit_models.zero_order import CartridgeFiltrationZO
from watertap.core.wt_database import Database
from watertap.core.zero_order_properties import WaterParameterBlock as ZOProperties
from watertap.costing.zero_order_costing import ZeroOrderCosting
from watertap.costing import WaterTAPCosting
from watertap.core.solvers import get_solver

In [ ]:
import idaes.logger as idaeslog
from idaes.core import declare_process_block_class
from idaes.core.util.exceptions import InitializationError
from idaes.models.unit_models.translator import TranslatorData


@declare_process_block_class("TranslatorZOtoSW")
class TranslatorZOtoSWData(TranslatorData):
    """
    Translator block for converting from ZO TDS to SW TDS
    """

    CONFIG = TranslatorData.CONFIG()

    def build(self):
        super().build()

        @self.Constraint(
            self.flowsheet().time,
            doc="Equality mass flow water equation",
        )
        def eq_flow_mass_rule(blk, t):
            return (
                blk.properties_out[t].flow_mass_phase_comp["Liq", "H2O"]
                == blk.properties_in[t].flow_mass_comp["H2O"]
            )

        @self.Constraint(
            self.flowsheet().time,
            doc="Equality solute equation",
        )
        def eq_solute_mass_flow(blk, t):
            return (
                blk.properties_out[t].flow_mass_phase_comp["Liq", "TDS"]
                == blk.properties_in[t].flow_mass_comp["TDS"]
            )

        # ZO prop pack doesn't have temperature and pressure as state variables
        self.properties_out[0].pressure.fix(101325)
        self.properties_out[0].temperature.fix(298.15)

    def initialize_build(
        self,
        state_args_in=None,
        state_args_out=None,
        outlvl=idaeslog.NOTSET,
        solver=None,
        optarg=None,
    ):
        init_log = idaeslog.getInitLogger(self.name, outlvl, tag="unit")

        # Create solver
        opt = get_solver(solver, optarg)

        # ---------------------------------------------------------------------
        # Initialize state block
        flags = self.properties_in.initialize(
            outlvl=outlvl,
            optarg=optarg,
            solver=solver,
            state_args=state_args_in,
            hold_state=True,
        )

        self.properties_out.initialize(
            outlvl=outlvl,
            optarg=optarg,
            solver=solver,
            state_args=state_args_out,
        )

        if degrees_of_freedom(self) != 0:
            raise Exception(
                f"{self.name} degrees of freedom were not 0 at the beginning "
                f"of initialization. DoF = {degrees_of_freedom(self)}"
            )

        with idaeslog.solver_log(init_log, idaeslog.DEBUG) as slc:
            res = opt.solve(self, tee=slc.tee)

        self.properties_in.release_state(flags=flags, outlvl=outlvl)

        init_log.info(f"Initialization Complete: {idaeslog.condition(res)}")

        if not check_optimal_termination(res):
            raise InitializationError(
                f"{self.name} failed to initialize successfully. Please check "
                f"the output logs for more information."
            )

# Problem Statement

Create the following flowsheet:

`feed > cartridge filtration > anti-scalant addition > pump > RO > product`

Additionally, the cartridge filtration waste stream and RO brine stream must end up mixed together in the same product block. Create an expression for the flux across the RO membranes in liters per m2 per hour (LMH).

I want to know:
- LCOW
- SEC
- CAPEX
- OPEX
- Pumping power required
- RO recovery
- TDS concentration in permeate
- TDS concentration in brine

# Model Parameters

In [ ]:
solute_list = ["TDS"]

mass_flow_water = 0.965
mass_flow_salt = 0.035

use_default_removal = True

# Chemical dosing parameters
anti_scalant_dose = 1 * pyunits.mg / pyunits.liter

# Pump parameters
pump_efficiency = 0.85 * pyunits.dimensionless
operating_pressure = 75 * pyunits.bar

# RO parameters
A_comp = 4.2e-12
B_comp = 3e-8
membrane_area = 50  # m2
atmospheric = 101325  # Pa
deltaP = -3 * pyunits.bar
channel_height = 1 * pyunits.mm
spacer_porosity = 0.75

solver = get_solver()

# Model Build

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)